# SDP and MBoot - Flashloader

This example shows how to download a flashloader into i.MX RT10xx device and read bootloader properties.

This example provides flashloader for i.MX RT1050. The binary was built using MCUXPresso IDE from  SDK mcu-boot middleware.

For more information read: https://www.nxp.com/docs/en/user-guide/MBOOTFLASHGS.pdf

In [1]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
#
# Copyright 2019-2023 NXP
#
# SPDX-License-Identifier: BSD-3-Clause

import os
from time import sleep
from typing import Optional

from spsdk.exceptions import SPSDKError
from spsdk.mboot.exceptions import McuBootError
from spsdk.mboot.interfaces.usb import MbootUSBInterface
from spsdk.mboot.mcuboot import McuBoot
from spsdk.sdp.exceptions import SdpError
from spsdk.sdp.interfaces.usb import SdpUSBInterface
from spsdk.sdp.sdp import SDP
from spsdk.utils.misc import load_binary

# Uncomment for printing debug messages
# import logging
# logging.basicConfig(level=logging.DEBUG)

FLASHLOADER = os.path.join("..", "_data", "ivt_flashloader.bin")


def run_flash_loader(
    load_address: int, start_address: int, data: bytes, device_name: Optional[str] = None
) -> bool:
    """Load an execute flashloader binary in i.MX-RT.

    :param load_address: Destination address in target memory
    :param start_address: Execution address
    :param data: flashloader binary data
    :param device_name: i.MX-RT device name or VID:PID
    :return: True if running flashloader was successful
    :raises sdp.SdpError: If SDP operation fails
    """
    interfaces = SdpUSBInterface.scan(device_id=device_name)
    if not interfaces:
        return False

    try:
        with SDP(interfaces[0], True) as serial_downloader:
            serial_downloader.write_file(load_address, data)
            serial_downloader.jump_and_run(start_address)
            return True
    except SdpError:
        return False


flashloader_data = load_binary(FLASHLOADER)

if run_flash_loader(0x20000000, 0x20000400, flashloader_data):
    sleep(6)  # wait for device startup
    print("flash-loader executed")

flash-loader executed


In [2]:
# Scan for MCU-BOOT device
interfaces = MbootUSBInterface.scan()
if not interfaces:
    raise SPSDKError("MCU-BOOT device not found")

try:
    with McuBoot(interfaces[0], True) as mb:
        # data = mb.read_memory(0, 500)
        property_list = mb.get_property_list()
        # mb.reset()

except McuBootError as e:
    raise SPSDKError(f"Error occurred:{str(e)}") from e

for prop in property_list:
    print(prop)

Current Version = K2.1.0
Available Peripherals = UART, USB-HID
Available Commands = ['FlashEraseAll', 'FlashEraseRegion', 'ReadMemory', 'WriteMemory', 'FillMemory', 'GetProperty', 'ReceiveSBFile', 'Execute', 'Call', 'Reset', 'SetProperty', 'FlashProgramOnce', 'FlashReadOnce', 'FlashReadResource', 'ConfigureMemory', 'GenerateKeyBlob']
Max Packet Size = 1016 B
Reserved Regions =
    Region 0: 0x20002000 - 0x20015FB7; Total Size: 79.9 kiB
    Region 1: 0x20208000 - 0x20215A6F; Total Size: 54.6 kiB
RAM Start Address = 0x00000000
RAM Size = 128.0 kiB
Security State = UNSECURE
Unique Device Identification = 84 24 F8 65 D2 39 1B 34
Target Version = T1.0.0
